In [334]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
from sklearn.linear_model import Lasso

import pysindy as ps
import pandas as pd
import numpy as np

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [335]:
from stesml.data_tools import get_scenario_index
from stesml.data_tools import get_train_and_val_index
from stesml.data_tools import load_data

In [336]:
data_dir = "../data/Sulfur_Models/heating/heating_all"

In [337]:
t_min = 1
t_max = 360

In [338]:
scenario_index = get_scenario_index(data_dir)

In [339]:
train_index, val_index = get_train_and_val_index(scenario_index, random_state=5)

In [340]:
train_data_ = load_data(scenario_index, train_index, t_min=t_min, t_max=t_max)
train_df = train_data_[['Tw','Ti','Tavg']]
train_df.index = train_data_['flow-time']
train_df

,Tw,Ti,Tavg
flow-time,,,
1.010680,540,400.0,400.874253
1.110680,540,400.0,400.920411
1.210680,540,400.0,400.964261
1.310680,540,400.0,401.006935
1.410680,540,400.0,401.048340
...,...,...,...
359.554681,460,400.0,404.503556
359.654681,460,400.0,404.504525
359.754681,460,400.0,404.505495


In [341]:
val_data_ = load_data(scenario_index, val_index, t_min=t_min, t_max=t_max)
val_df = val_data_[['Tw','Ti','Tavg']]
val_df.index = val_data_['flow-time']
val_df

,Tw,Ti,Tavg
flow-time,,,
1.066673,560,440.0,440.778176
1.166673,560,440.0,440.817492
1.266673,560,440.0,440.855069
1.366673,560,440.0,440.891111
1.466673,560,440.0,440.925782
...,...,...,...
359.669535,520,440.0,446.627400
359.769535,520,440.0,446.627971
359.869535,520,440.0,446.628541


In [342]:
train_data = list()
train_index = list()
for idx, grp in train_df.groupby(["Tw", "Ti"]):
    train_data.append(grp.values)
    train_index.append(grp.index.values)

In [343]:
val_data = list()
val_index = list()
for idx, grp in val_df.groupby(["Tw", "Ti"]):
    val_data.append(grp.values)
    val_index.append(grp.index.values)

In [386]:
poly_library = ps.feature_library.PolynomialLibrary(degree=5)
optimizer = ps.STLSQ(threshold=0,normalize_columns=True,verbose=True,alpha=0)
model = ps.SINDy(optimizer=optimizer, feature_names=['Tw','Ti','Ts'], feature_library=poly_library)

In [387]:
model.fit(train_data, t=train_index, multiple_trajectories=True)

 Iteration ... |y - Xw|^2 ...  a * |w|_2 ...      |w|_0 ... Total error: |y - Xw|^2 + a * |w|_2
         0 ... 1.4155e+02 ... 0.0000e+00 ...        168 ... 1.4155e+02


SINDy(differentiation_method=FiniteDifference(),
      feature_library=PolynomialLibrary(degree=5),
      feature_names=['Tw', 'Ti', 'Ts'],
      optimizer=STLSQ(alpha=0, normalize_columns=True, threshold=0,
                      verbose=True))

In [388]:
model.print(precision=5)

(Tw)' = 0.00000
(Ti)' = 0.00000
(Ts)' = -0.00042 Tw + -0.00041 Ti + -0.00041 Ts + 0.00026 Tw^2 + -0.00068 Tw Ti + -0.00001 Tw Ts + -0.00026 Ti^2 + 0.00014 Ti Ts + 0.00056 Ts^2 + 0.00007 Tw^2 Ti + -0.00008 Tw^2 Ts + 0.00068 Tw Ti^2 + -0.00150 Tw Ti Ts + 0.00083 Tw Ts^2 + -0.00043 Ti^3 + 0.00054 Ti^2 Ts + 0.00030 Ti Ts^2 + -0.00041 Ts^3 + -0.00001 Tw^2 Ti^2 + 0.00002 Tw^2 Ti Ts + -0.00001 Tw^2 Ts^2 + -0.00010 Tw Ti^3 + 0.00030 Tw Ti^2 Ts + -0.00032 Tw Ti Ts^2 + 0.00011 Tw Ts^3 + -0.00038 Ti^4 + 0.00161 Ti^3 Ts + -0.00259 Ti^2 Ts^2 + 0.00184 Ti Ts^3 + -0.00049 Ts^4 + -0.00001 Tw Ti^3 Ts + 0.00002 Tw Ti^2 Ts^2 + -0.00001 Tw Ti Ts^3 + 0.00001 Ti^5 + -0.00003 Ti^4 Ts + 0.00006 Ti^3 Ts^2 + -0.00007 Ti^2 Ts^3 + 0.00004 Ti Ts^4 + -0.00001 Ts^5


In [389]:
n_cases = len(val_data)

In [ ]:
predictions = list()
for i in range(n_cases):
    prediction = model.simulate(x0=np.array(val_data[i][0]).reshape(3),t=val_index[i])
    predictions.append(prediction)

In [ ]:
for i in range(n_cases):
    plt.plot(val_index[i], val_data[i][:,2] - predictions[i][:,2])
    plt.xlim(0,360)

In [ ]:
predictions_joined = list()
val_data_joined = list()

for i in range(n_cases):
    val_data_joined += val_data[i][:,2].tolist()
    predictions_joined += predictions[i][:,2].tolist()

In [ ]:
r2 = r2_score(val_data_joined, predictions_joined)
rmse = mean_squared_error(val_data_joined, predictions_joined, squared=False)
print(f'RMSE: {rmse}, R2: {r2}')

In [ ]:
plt.style.use('ggplot')
for i in range(len(val_data_list)):
    Tw = val_data_list[i][:,0][0]
    Ti = val_data_list[i][:,1][0]
    plt.plot(val_index[i], val_data[i][:,2], linewidth=2.5, c='DarkBlue', label='Expected')
    plt.plot(val_index[i], predictions[i][:,2], linewidth=2.5, c='DarkOrange', label='Predicted')
    plt.xlabel('flow-time')
    plt.title(f'Tw = {Tw}  Ti = {Ti}')
    plt.legend()
    plt.show()